# ML projet - Groupe 3

# Pretraitement

In [33]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import enchant

#nltk.download('punkt_tab')
#nltk.download("words")
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger_eng')

In [34]:
df = pd.read_csv("scitweets_export.tsv", sep = "\t")

In [35]:
len(df[df["science_related"] == 0])/(len(df))

0.6710526315789473

In [36]:
#df[df["tweet_id"].duplicated()]

In [37]:
#df[df["text"].duplicated()]

In [38]:
#df[df["science_related"] == 0]

### Enlever (.,\\@#:';"“”’[]), les chiffres et les liens (http) de chaque tweet

In [39]:
text = []
all_text = list(df["text"])
for t in all_text:
    a = re.sub("[\.,/\\@:;\"\'0-9“”’\[\]]", "", t)
    a = re.sub("http\S*", "", a) #Enlever les liens
    a = re.sub("#\S*", "", a) #Enlever les tags
    a = a.lower()
    text.append(a)
#for sentence in text:  

In [40]:
#text

In [41]:
#Créer une liste des phrases ou chaque phrase est une liste de mots.
text_sep = [word_tokenize(t) for t in text]

In [42]:
#text_sep

## Standardizer chaque phrases. 
- Convertir les noms pluriels en leur forme singuliers
- Convertir les conjugaison de chaque verbe en leur infinitif
- Standardiser les ajectives et les adverbes

In [43]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):  # Adjectif
        return wordnet.ADJ
    elif tag.startswith('V'):  # Verbe
        return wordnet.VERB
    elif tag.startswith('N'):  # Nom
        return wordnet.NOUN
    elif tag.startswith('R'):  # Adverbe
        return wordnet.ADV
    else:
        return wordnet.NOUN 

    
lemmatizer = WordNetLemmatizer()
refined_tweets = []
for tweet in text_sep:
    pos_tags = pos_tag(tweet)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    refined_tweets.append(lemmatized_words)



In [44]:
ind = 640
#print(text_sep[ind])
#print(refined_tweets[ind])

### Le code desous montre que 70% de mots dans les tweet sont valide (dans les dictionnaire anglais)

In [45]:
valid_words = []
invalid_words = []

d = enchant.Dict("en_US")

#Trouver tous les mots qui sont dans la dictionnaiore anglais 
for sentence in refined_tweets:
    for w in sentence:
        if w not in valid_words:
            if d.check(w):
                valid_words.append(w)
            if d.check(w.capitalize()) and w not in valid_words:
                valid_words.append(w)
                
            
total_words = []
for sentence in refined_tweets:
    for w in sentence:
        if w not in total_words:
            total_words.append(w)

            
print(len(valid_words)/len(total_words))


0.7057108557708798


In [46]:
invalid_words = []
for sentence in refined_tweets:
    for w in sentence:
        if w not in invalid_words:
            if w not in valid_words:
                invalid_words.append(w)
#invalid_words

### Vectoriser les mots

In [48]:
model = Word2Vec(refined_tweets, vector_size = 200, window = 2, min_count = 0, workers = 4)

In [49]:
(model.wv["science"])

array([ 1.14255445e-03, -1.61119050e-03, -1.41837390e-03,  3.42491479e-03,
        1.13822361e-02, -1.25815405e-03,  1.89015525e-03,  2.12700814e-02,
       -2.44117714e-03,  4.23940411e-03, -2.70634028e-03, -4.91850823e-03,
        6.38018455e-03,  7.86870997e-03, -4.59962850e-03, -3.72354261e-04,
       -1.49839465e-03,  5.13493223e-03,  5.39186224e-03, -1.05113965e-02,
        8.21312424e-03, -1.10082084e-03,  9.88406246e-04, -3.41620459e-03,
        1.58569671e-03, -8.56285263e-03,  2.66132574e-03, -3.73570039e-03,
       -1.04353353e-02, -3.10109439e-03,  9.01723653e-03,  1.33899297e-03,
        5.38798235e-03, -2.25112541e-03,  1.87710626e-03,  6.74281362e-03,
        3.86751909e-03,  3.90663696e-03,  2.48991163e-03, -8.87915865e-03,
       -2.43932521e-03, -3.17847589e-03, -3.72511242e-03, -1.61992991e-03,
        8.41878634e-03, -1.03804225e-03, -5.67010231e-03, -3.08961887e-03,
        4.98298206e-04,  1.16749713e-02,  3.04161245e-03, -3.51572828e-03,
       -6.89601433e-03, -